### 数据集
- 我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。
- 一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科的合集上预训练的，但它很难吸引这本书的大多数读者。
- 另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。
- 为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2。
- 与用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。

In [423]:
import os
import random
import torch
from d2l import torch as d2l

- 在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。
- 保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。

In [424]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

可以看出文本的加载形式，文本被加载成列表的形式，每个列表又是一个储存了多个句子的列表.
里面的为止部分已经替换成了unk

In [425]:
data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
paragraphs = _read_wiki(data_dir)
paragraphs[2]

["having damaged his knee ligaments during the play @-@ off semi @-@ final , cresswell missed the start of the 2006 – 07 season , making his first appearance in leeds ' 4 – 0 home defeat to stoke city on 14 october 2006",
 "however , he sustained a knee injury in november 2006 , after scoring his first goal of the campaign in leeds ' 3 – 0 home win over colchester united on 11 november",
 'his return from injury came as a 68th @-@ minute substitute in a 2 – 1 victory at home to crystal palace on 10 february 2007',
 'cresswell scored in successive games against sheffield wednesday and luton town in march 2007 , but leeds were eventually relegated to league one',
 'he finished the season with four goals in 23 games .']

接下来是bert所需的输入数据：
- bert和以往模型不一样的地方在于，有非常多的输入数据
- 1、和其他模型一样，语料库相关数据，
- 2、segment，对于句子对进行前后句子区分
- 3、句子有效长度
- 4、掩码矩阵权重、掩码信息
- 5、下一个句子预测训练样本

先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

_get_next_sentence函数生成二分类任务的训练样本。

In [426]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    """这个函数的目的就是在于生成二分类的任务
    输入一个句子sentence,以及其真实的下一个句子
    有一半概率, 我接下来的句子就是真实句子
    但有有一半概率我把这个真实的next_sentence丢掉, 
    直接从paragraphs里面选择一个句子来作为next_sentence
    那么我就构建好了这么一个预测模型:
        如果next_sentence确实是,那么is_next就是True
        反之is_next就是False, 所以is_next就是标签
    """
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

- 下面的函数通过调用_get_next_sentence函数从输入paragraph生成用于下一句预测的训练样本。
- 这里paragraph是句子列表，其中每个句子都是词元列表。
- 自变量max_len指定预训练期间的BERT输入序列的最大长度。

In [427]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        # 这里把每个句子
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # 这里生成的tokens, segments就可以作为bert的输入了
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        # 这里是构建下一个句子预测的样本,这里可以看到放入的是tokens, segments,而不是token_a, token_b
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

演示一下如何使用

In [428]:
paragraphs = [d2l.tokenize(
    paragraph, token='word') for paragraph in paragraphs]

此时paragraphs是将整个文章彻底拆开

In [429]:
paragraphs[2][0]

['having',
 'damaged',
 'his',
 'knee',
 'ligaments',
 'during',
 'the',
 'play',
 '@-@',
 'off',
 'semi',
 '@-@',
 'final',
 ',',
 'cresswell',
 'missed',
 'the',
 'start',
 'of',
 'the',
 '2006',
 '–',
 '07',
 'season',
 ',',
 'making',
 'his',
 'first',
 'appearance',
 'in',
 'leeds',
 "'",
 '4',
 '–',
 '0',
 'home',
 'defeat',
 'to',
 'stoke',
 'city',
 'on',
 '14',
 'october',
 '2006']

In [430]:

sentences = [sentence for paragraph in paragraphs
                for sentence in paragraph]

sentences是把所有句子全部拉成一个列表，不再区分paragraph

In [431]:
sentences[12]

['his',
 'return',
 'from',
 'injury',
 'came',
 'as',
 'a',
 '68th',
 '@-@',
 'minute',
 'substitute',
 'in',
 'a',
 '2',
 '–',
 '1',
 'victory',
 'at',
 'home',
 'to',
 'crystal',
 'palace',
 'on',
 '10',
 'february',
 '2007']

In [432]:
vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])

In [433]:
for paragraph in paragraphs:
    sample = _get_nsp_data_from_paragraph(paragraph,paragraphs,vocab,64)
    break

可以看到，_get_nsp_data_from_paragraph将句子处理成了[[sentence]，[segment]]形式

In [434]:
sample[0][0][:10]

['<cls>',
 'the',
 'first',
 'sequel',
 'to',
 'species',
 ',',
 'species',
 'ii',
 'was']

In [435]:
sample[0][1][30:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### 生成遮蔽语言模型任务的数据
- 为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下_replace_mlm_tokens函数。
- 在其输入中，tokens是表示BERT输入序列的词元的列表，candidate_pred_positions是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及num_mlm_preds指示预测的数量（选择15%要预测的随机词元）。在 14.8.5.1节中
- 定义遮蔽语言模型任务之后，在每个预测位置，输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。

In [436]:
"""
BERT预训练任务中被选中的15%的词中仅80%的词是真正被Masked掉的,另外10%不变，最后10%随机换为其他词，
"""
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    """
    后面代码显示 num_mlm_preds = max(1, round(len(tokens) * 0.15)) 
    num_mlm_preds为整体token长度的15%，至少为1
    
    candidate_pred_positions是把tokens里面不是'<cls>', '<sep>'的词元索引拿出来
    
    """
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        # num_mlm_preds用于控制百分之多少的词用于替换
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        ##################################################
        # 下面这部分是按比例去生成masked_token
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        #################################################    
        #  mlm_pred_position是candidate_pred_positions中选出是索引
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    # mlm_input_tokens：token替换后的token_list, pred_positions_and_labels :待预测索引和标签
    return mlm_input_tokens, pred_positions_and_labels

In [437]:
batch_size, max_len = 512, 64
for tokens, segments, is_next in _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    print(tokens,end='\n')
    print(segments,end='\n')
    print(is_next,end='\n')
    break

['<cls>', 'the', 'first', 'sequel', 'to', 'species', ',', 'species', 'ii', 'was', 'released', 'theatrically', 'in', 'april', '1998', '<sep>', 'the', 'film', 'depicts', 'astronauts', 'on', 'a', 'mission', 'to', 'mars', 'being', 'attacked', 'by', 'the', 'aliens', 'from', 'species', ',', 'and', 'the', 'events', 'that', 'ensue', 'upon', 'their', 'return', 'to', 'earth', '<sep>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
True


In [438]:
candidate_pred_positions = []
# tokens是一个字符串列表
for i, token in enumerate(tokens):
    # 在遮蔽语言模型任务中不会预测特殊词元
    if token in ['<cls>', '<sep>']:
        continue
    candidate_pred_positions.append(i)
num_mlm_preds = max(1, round(len(tokens) * 0.15))

- 函数演示,和上面原始的tokens对比，可以看到mlm_input_tokens已经把部分词替换成了mask，部分词进行了随机替换
- pred_positions_and_labels储存的是你具体吧哪个位置进行了替换，替换前是啥样
  - 比如(14, 'under')就是把第14个的'under'替换成了mask

In [439]:
mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens,candidate_pred_positions,num_mlm_preds,vocab)
print(mlm_input_tokens)
print(pred_positions_and_labels)

['<cls>', '<mask>', 'first', 'sequel', 'to', 'species', '<mask>', '<mask>', 'ii', 'was', 'released', 'theatrically', 'in', 'april', '1998', '<sep>', 'the', 'recordings', 'depicts', 'astronauts', 'on', 'a', 'mission', 'to', 'mars', 'being', 'attacked', 'by', 'the', '<mask>', '<mask>', 'species', ',', 'and', 'the', 'events', 'that', 'ensue', 'upon', 'their', 'return', 'to', 'earth', '<sep>']
[(29, 'aliens'), (17, 'film'), (7, 'species'), (6, ','), (25, 'being'), (30, 'from'), (1, 'the')]


通过调用前述的_replace_mlm_tokens函数，以下函数将BERT输入序列（tokens）作为输入，并返回输入词元的索引、发生预测的词元索引以及这些预测的标签索引。

In [440]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    # 输入词元的索引、发生预测的词元索引以及这些预测的标签索引。
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

依旧是演示
- vocab[mlm_input_tokens]返回的是vocab里面每个token的索引
- pred_positions就是替换的位置
- vocab[mlm_pred_labels]是具体替换的词在vocab的索引

In [441]:
pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
pred_positions_and_labels

[(1, 'the'),
 (6, ','),
 (7, 'species'),
 (17, 'film'),
 (25, 'being'),
 (29, 'aliens'),
 (30, 'from')]

### 将文本转换为预训练数据集

BERT预训练定制一个Dataset类。在此之前，我们仍然需要定义辅助函数_pad_bert_inputs来将特殊的“<mask>”词元附加到输入。它的参数examples包含来自两个预训练任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。

In [442]:
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

先看看example长啥样

In [452]:
examples = []
for paragraph in paragraphs:
    examples.extend(_get_nsp_data_from_paragraph(
        paragraph, paragraphs, vocab, max_len))
# 获取遮蔽语言模型任务的数据
examples = [(_get_mlm_data_from_tokens(tokens, vocab)
                + (segments, is_next))
                for tokens, segments, is_next in examples]

看到这里，我们总算可以对目前的处理进行总结，李沐大佬把目前的所有处理数据整合成了一个example，方便后续进行计算，一个example对应一个tokens

可以看出一个标准example是包含了_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的所有东西，包括如下东西
- _get_nsp_data_from_paragraph的返回：tokens, segments, is_next
- _get_mlm_data_from_tokens 的返回：vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]
- 也就是说包含如下信息
  - tokens:句子对
    - 注意下，本代码的定义中：tokens = ['<cls>'] + tokens_a + ['<sep>']， tokens是句子对而非vocab里面的tokens
  - segments：句子对信息
  - is_next：下一个句子训练样本
  - vocab[mlm_input_tokens]：15%选出的token对于语料库里面的索引
  - pred_positions：这15%token里面，被屏蔽或者被替换的索引位置（在tokens里面的索引）
  - vocab[mlm_pred_labels]：这被替换的是哪些词

In [443]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

In [444]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers() # 4
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [445]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


tokens编码

In [446]:
tokens_X[:2]

tensor([[    3,   313,  6552,  1075,   196,    10,    32,     2,     0,     0,
           840,   413,     6,     8,    22,   203,     2,     6, 11283,   126,
            11,   352,     4,  7398,   352,   140,    10,     2,    10,     5,
           115,    14,  1541,     9,     2,    10,    11,  5538,   193,     7,
             5,   192,   578,     2,     5,   134,   155,     4,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    3,     5,  7364,  2409,    23,    11,   185,     2,  1621,    10,
           674,   993,    25,   384,    55,   349,     9,    24,   166,     6,
          8153,  1182,     4,    86,    36,   129,   207,   221,     2, 17782,
         10175,  4876,   711, 14831,    25,     5,   701,     2,     6,     2,
         11123,     2,   711, 13755,     8,   711, 11318,     4,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1

segment编码，表示是第几个句子

In [447]:
segments_X[:2]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

每个句子有效长度

In [448]:
valid_lens_x[:100]

tensor([48., 48., 41., 41., 55., 52., 55., 57., 25., 54., 29., 38., 54., 57.,
        47., 47., 36., 36., 47., 46., 59., 40., 38., 59., 50., 62., 51., 55.,
        64., 62., 58., 52., 48., 34., 57., 58., 33., 40., 63., 57., 38., 40.,
        54., 57., 63., 35., 56., 63., 41., 45., 38., 39., 54., 37., 51., 40.,
        48., 62., 58., 61., 45., 55., 41., 53., 62., 63., 44., 36., 35., 44.,
        35., 52., 57., 51., 48., 55., 63., 36., 59., 53., 44., 21., 52., 30.,
        50., 62., 58., 28., 47., 36., 58., 37., 56., 60., 15., 63., 40., 54.,
        60., 53.])

待预测句子的位置编码

In [449]:
pred_positions_X[100:110]

tensor([[ 9, 10, 24, 26, 27, 30,  0,  0,  0,  0],
        [ 5,  8, 12, 16, 27, 32, 34, 36, 40,  0],
        [ 2,  6, 12, 20, 23, 30,  0,  0,  0,  0],
        [ 3,  4,  6,  9, 18, 22, 25, 33,  0,  0],
        [ 1,  3,  8,  9, 10, 30, 34,  0,  0,  0],
        [ 2,  9, 12, 23, 24, 29, 30,  0,  0,  0],
        [ 1,  6, 19, 24, 29, 33, 40, 50,  0,  0],
        [ 2, 14, 16, 17, 24, 35,  0,  0,  0,  0],
        [ 3,  5, 20, 21, 30, 31, 34, 47, 50,  0],
        [ 3,  4, 15, 27, 30,  0,  0,  0,  0,  0]])

掩码矩阵初试权重

In [450]:
mlm_weights_X[:10]

tensor([[1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.]])

掩码位置

In [451]:
mlm_Y[:10]

tensor([[    0,   118,     5,   140,   623,  1001,     7,     0,     0,     0],
        [    6,     9,     5,    54,    24,   711,     6,     0,     0,     0],
        [   56,   131,     5,  3499,     6,  8116,     0,     0,     0,     0],
        [   40,    10,  1069,  1877,  1246,   899,     0,     0,     0,     0],
        [  652,     8,     0,   335, 16945,   983,    91,    12,     0,     0],
        [  140,    87,     9,    11,    10,  2228,     6,    49,     0,     0],
        [  228,   702,  2747,    53,    28,    17,     9,  2573,     0,     0],
        [    0,    25,    12,  2600,    19,     6,     8,   496,  1561,     0],
        [  147,    19,    17,    12,     0,     0,     0,     0,     0,     0],
        [  165,  2894,     6,     5,    55,    91,    25,    22,     0,     0]])